In [ ]:
'''
Example of tensorflow(keras)-based distance driven 2d forward and backprojection
'''

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
import SimpleITK as sitk
import tensorflow as tf
import os
import tensorflow.keras.backend as K

In [ ]:
import ct_projector.projector.tensorflow as ct_projector_tf
import ct_projector.projector.tensorflow.circular_2d as ct_circular_tf
import ct_projector.projector.tensorflow.filters as ct_filters_tf

In [ ]:
os.environ['CUDA_VISIBLE_DEVICES'] = '0'
physical_devices = tf.config.list_physical_devices('GPU')
tf.config.experimental.set_memory_growth(physical_devices[0], True)

In [ ]:
# load a sample CT image
filename = './3.nii.gz'
ct = sitk.ReadImage(filename)
spacing = ct.GetSpacing()
img = sitk.GetArrayFromImage(ct)

# convert image from HU to attenuation coefficient
# This is the approximate relationship
img = (img.astype(np.float32) + 1024) / 1000 * 0.019
img[img < 0] = 0

img = img[200:232]

# also convert to image to our projector dimension batch, z, y, x
img = img[::-1, ...]
spacing = np.array(spacing[::-1])

vmin = -350 / 1000 * 0.019
vmax = 1150 / 1000 * 0.019

In [ ]:
# show the ct images
plt.figure(figsize = (12,4))
plt.subplot(131); plt.imshow(img[img.shape[0]//2, ...], 'gray', aspect=spacing[1] / spacing[2], vmin=vmin, vmax=vmax)
plt.subplot(132); plt.imshow(img[:, img.shape[1]//2, :], 'gray', aspect=spacing[0] / spacing[2], vmin=vmin, vmax=vmax)
plt.subplot(133); plt.imshow(img[..., img.shape[2]//2], 'gray', aspect=spacing[0] / spacing[1], vmin=vmin, vmax=vmax)

The projector holds the default parameters for the forward and backprojection. 

However, all the projection parameters are passed during calculation to enable training with various geometry. The only constraint is that within one batch the input images must be of the same shape, otherwise they cannot be passed as tensors `[batch, nx, ny, nz, channel]`. The output images must also have the same shapes. 

In [ ]:
# setup the projector
projector = ct_projector_tf.ct_projector()
projector.from_file('./projector_fan.cfg')
projector.nx = img.shape[2]
projector.ny = img.shape[1]
projector.nz = 1
projector.dx = spacing[2]
projector.dy = spacing[1]
projector.dz = spacing[0]
projector.nv = 1
projector.nview = 768

angles = projector.get_angles()

for k in vars(projector):
    print (k, '=', getattr(projector, k))

In [ ]:
img_input = img[:, np.newaxis, :, :, np.newaxis]
input_tensor = tf.convert_to_tensor(img_input, tf.float32)
print(input_tensor.shape)

fp_model = ct_circular_tf.DistanceDriven2DFP(
    projector,
    angles,
    ct_circular_tf.TypeGeometry.PARALLEL,
    ct_circular_tf.TypeProjector.IR,
)

bp_model = ct_circular_tf.DistanceDriven2DBP(
    projector,
    angles,
    ct_circular_tf.TypeGeometry.PARALLEL,
    ct_circular_tf.TypeProjector.FORCE_FBP,
)

filter_model = ct_filters_tf.ProjectionFilter(
    projector.du,
    ct_circular_tf.TypeGeometry.PARALLEL,
    ct_filters_tf.TypeFilter.RL
)


In [ ]:
with tf.GradientTape() as t:
    t.watch(input_tensor)
    fp_tensor = fp_model(input_tensor)
    loss = tf.reduce_sum(fp_tensor * fp_tensor / 2)
grad_tensor = t.gradient(loss, input_tensor)

fprj_tensor = filter_model(fp_tensor)
bp_tensor = bp_model(fprj_tensor)

print(fp_tensor.shape)
print(bp_tensor.shape)
print(grad_tensor.shape)

In [ ]:
fp = fp_tensor.numpy()
plt.figure()
plt.imshow(fp[fp.shape[0] // 2, :, 0, :, 0], 'gray', vmin=0, vmax=10)

bp = bp_tensor.numpy()
plt.figure()
plt.imshow(bp[bp.shape[0] // 2, 0, :, :, 0], 'gray', vmin=vmin, vmax=vmax)

In [ ]:
grad = grad_tensor.numpy()
print(np.abs(grad - bp).max())
plt.imshow(grad[grad.shape[0] // 2, 0, :, :, 0], 'gray')

In [ ]:
import cupy as cp
from ct_projector.projector.cupy.parallel import distance_driven_fp, distance_driven_bp, ramp_filter

cuimg = cp.array(img, order='C')[:, cp.newaxis, :, :]
cuangles = cp.array(angles, order='C')
cufp_ref = distance_driven_fp(projector, cuimg, cuangles)

fp_ref = cufp_ref.get()
fp_ref = fp_ref[..., np.newaxis]
plt.figure()
plt.imshow(fp_ref[fp_ref.shape[0] // 2, :, 0, :, 0], 'gray', vmin=0, vmax=10)

cufprj_ref = ramp_filter(projector, cufp_ref, 'RL')

cubp_ref = distance_driven_bp(projector, cufprj_ref, cuangles, is_fbp=True)
bp_ref = cubp_ref.get()
bp_ref = bp_ref[..., np.newaxis]
plt.figure()
plt.imshow(bp_ref[bp_ref.shape[0] // 2, 0, :, :, 0], 'gray', vmin=vmin, vmax=vmax)

plt.figure()
plt.imshow((bp - bp_ref)[bp_ref.shape[0] // 2, 0, :, :, 0], 'gray', vmin=-0.001, vmax=0.001)

print(np.abs(fp - fp_ref).max())
print(np.abs(bp - bp_ref).max())

In [ ]:
# test training
fp_input = fp[:, :, 0, :, :]
label = img_input * 1.1
model_input = tf.keras.layers.Input(shape=[fp.shape[1], fp.shape[3], 1])
x = tf.keras.layers.Conv2D(1, 1, padding='same', use_bias=False)(model_input)
x = x[:, :, tf.newaxis, :, :]
x = filter_model(x)
x = bp_model(x)

model = tf.keras.Model(inputs=model_input, outputs=x)
optimizer = tf.keras.optimizers.Adam(0.01)
loss = tf.keras.losses.MeanSquaredError()

model.compile(optimizer, loss)

In [ ]:
model.fit(fp_input, label, batch_size=4, epochs=100)

In [ ]:
pred = model.predict(fp_input)

In [ ]:
plt.figure()
plt.imshow(label[label.shape[0] // 2, 0, :, :, 0], 'gray', vmin=vmin, vmax=vmax)

plt.figure()
plt.imshow(pred[pred.shape[0] // 2, 0, :, :, 0], 'gray', vmin=vmin, vmax=vmax)

In [ ]:
model.layers[1].get_weights()